<a href="https://colab.research.google.com/github/cph316/generative_ai/blob/main/%E3%80%90HW7%E3%80%910401_%E7%94%A8Ollama_%E6%89%93%E9%80%A0%E8%87%AA%E5%B7%B1%E7%9A%84%E5%B0%8D%E8%A9%B1%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Ollama](https://ollama.com/) 可以讓我們在自己的機器上跑開源的大型語言模型, 並且用 API 的方式呼叫。這裡我們介紹在 Colab 上跑, 並且分別用 OpenAI 的 API, 及 [`aisuite` 套件](https://github.com/andrewyng/aisuite) 來使用 Ollama 提供的大型語言模型。

### 1. 安裝並執行 Ollama

In [ ]:
!curl -fsSL https://ollama.ai/install.sh | sh

# 需要跑 Ollama Server, 這裡要求放在背景執行
!nohup ollama serve &

#  Llama 3.2 4B 模型
!ollama pull gemma3:4b

### 2. 用 OpenAI API 使用

In [ ]:
import openai
from openai import OpenAI

api_key = "ollama"

# 如同一般 OpenAI API 打開 client 的方式, 只是這裡多了 API 服務的網址。注意在自己家 (事實上是 Google Colab 的機器), 預設服務 port 是 11434。
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 3. 打造一個對話機器人 web app!

In [ ]:
!pip install gradio

import gradio as gr

In [ ]:
title = "旅遊美食小幫手"
system = "你是一位熱情又專業的旅遊導覽機器人，擅長推薦景點與當地美食，語氣親切，彷彿像朋友一樣陪伴旅行，回答請盡量使用臺灣習慣的繁體中文。"
description = "嗨～我是你的旅遊美食小幫手 🍱✈️ 想去哪裡玩？我可以幫你推薦景點、美食，還有當地的小故事唷！"
model = "gemma3:4b"

initial_messages = [{"role": "system", "content": system},
           {"role": "assistant", "content": description}]

state = gr.State(initial_messages)

In [ ]:
def travel_bot(prompt, messages):
    # 判斷是否為結束對話指令
    if prompt.strip().lower() in ["bye", "exit", "quit", "再見", "掰掰", "bye bye"]:
        farewell = "謝謝你的提問～祝你旅途愉快，有緣再見囉！👋"
        messages.append({"role": "user", "content": prompt})
        messages.append({"role": "assistant", "content": farewell})
        return messages, gr.update(visible=False)  # 關閉 chat 輸出

    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
    )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    return messages, messages

In [ ]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## 🤖 {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="輸入訊息")
    state = gr.State(initial_messages.copy())  # 務必用 copy()

    msg.submit(fn=travel_bot, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)